<a href="https://colab.research.google.com/github/pareshmishra23/SWIFT-MT564-AI/blob/main/Gemma_QLoRAMT564.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 🛠️ Step 1: Install Required Libraries
!pip install -q transformers datasets peft accelerate bitsandbytes

# ✅ Restart kernel after install if running for the first time


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.3 MB/s eta 0:00:00


In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
# 📦 Step 2: Import Libraries
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
import json


In [2]:
from google.colab import files
uploaded = files.upload()  # Upload your mt564_autotrain_text.jsonl

Saving mt564_autotrain_text_jsnl.jsonl to mt564_autotrain_text_jsnl.jsonl


In [3]:
# 📂 Step 3: Load Instruction-Tuned Data (.jsonl)
# Format: {"text": "### Instruction:\n...\n### Response:\n..."}
with open("mt564_autotrain_text_jsnl.jsonl") as f:
    lines = [json.loads(line) for line in f]

dataset = Dataset.from_list(lines)


In [4]:
from huggingface_hub import login

login(token="hugging face token  ")

In [5]:
# 🤖 Step 4: Load Tokenizer & Quantized Model (QLoRA Config)
model_id = "google/gemma-2b-it"  # Prefer 2B for Colab

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token  # Important!
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
# 🔁 Step 5: Apply LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none"
)
model = get_peft_model(model, peft_config)


In [7]:
# ✂️ Step 6: Tokenize Dataset
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/82 [00:00<?, ? examples/s]

In [8]:
# 🏋️ Step 7: Training Arguments
training_args = TrainingArguments(
    output_dir="./mt564-gemma-model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    learning_rate=3e-5,
    save_total_limit=1,
    bf16=False,
    fp16=True,
    report_to="none"
)


In [9]:
# 🚀 Step 8: Initialize Trainer and Start Training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,4.201400
20,3.921500
30,3.998700


TrainOutput(global_step=33, training_loss=4.040726950674346, metrics={'train_runtime': 82.0677, 'train_samples_per_second': 2.998, 'train_steps_per_second': 0.402, 'total_flos': 1498430299963392.0, 'train_loss': 4.040726950674346, 'epoch': 3.0})

In [10]:
# Save to local folder
model.save_pretrained("./mt564-gemma-lora")
tokenizer.save_pretrained("./mt564-gemma-lora")

# Push to Hugging Face (make sure you’re logged in)
model.push_to_hub("pareshmishra/mt564-gemma-lora")
tokenizer.push_to_hub("pareshmishra/mt564-gemma-lora")



adapter_model.safetensors:   0%|          | 0.00/3.70M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pareshmishra/mt564-gemma-lora/commit/8ab301789ade871a9b5a344c4775f900dd7cd024', commit_message='Upload tokenizer', commit_description='', oid='8ab301789ade871a9b5a344c4775f900dd7cd024', pr_url=None, repo_url=RepoUrl('https://huggingface.co/pareshmishra/mt564-gemma-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='pareshmishra/mt564-gemma-lora'), pr_revision=None, pr_num=None)

In [13]:
prompt = """### Instruction:
Analyze this MT564 message for anomalies

### Input:
{1:F01TESTBANKXXXX0000000000}{2:I564CLIENTBANKXXXXN}{4:
:16R:GENL
:20C::CORP//CA20250501
:23G:NEWM
:22F::CAEV//DVCA
:16S:GENL
:16R:CAOPTN
:13A::CAON//001
:36B::ENTL//UNIT/5000000
:19A::SETT//ZAR/5000000
:95Q::RCPT//KP
:16S:CAOPTN
}

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=300)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Instruction:
Analyze this MT564 message for anomalies

### Input:
{1:F01TESTBANKXXXX0000000000}{2:I564CLIENTBANKXXXXN}{4:
:16R:GENL
:20C::CORP//CA20250501
:23G:NEWM
:22F::CAEV//DVCA
:16S:GENL
:16R:CAOPTN
:13A::CAON//001
:36B::ENTL//UNIT/5000000
:19A::SETT//ZAR/5000000
:95Q::RCPT//KP
:16S:CAOPTN
}

### Response:
{1:F01TESTBANKXXXX0000000000}{2:I564CLIENTBANKXXXXN}{4:
:16R:GENL
:20C::CORP//CA20250501
:23G:NEWM
:22F::CAEV//DVCA
:16S:GENL
:16R:CAOPTN
:13A::CAON//001
:36B::ENTL//UNIT/5000000
:19A::SETT//ZAR/5000000
:95Q::RCPT//KP
:16S:CAOPTN}

### Anomalies:

Sure, here are the anomalies in the message:

* **Timestamp:** The timestamp is not provided in the message, which makes it impossible to determine the time of day the transaction took place.
* **Sender and receiver bank IDs:** The sender and receiver bank IDs are not provided in the message, which makes it impossible to verify the authenticity of the transaction.
* **Transaction type:** The transaction type is not provided in the 

In [11]:
!ls -lh /content/mt564-gemma-lora


total 41M
-rw-r--r-- 1 root root  823 Jul 15 21:33 adapter_config.json
-rw-r--r-- 1 root root 3.6M Jul 15 21:33 adapter_model.safetensors
-rw-r--r-- 1 root root  591 Jul 15 21:33 chat_template.jinja
-rw-r--r-- 1 root root 5.1K Jul 15 21:33 README.md
-rw-r--r-- 1 root root  522 Jul 15 21:33 special_tokens_map.json
-rw-r--r-- 1 root root  40K Jul 15 21:33 tokenizer_config.json
-rw-r--r-- 1 root root  33M Jul 15 21:33 tokenizer.json
-rw-r--r-- 1 root root 4.1M Jul 15 21:33 tokenizer.model


to download locally from colab

In [12]:
!zip -r mt564-gemma-lora.zip mt564-gemma-lora
from google.colab import files
files.download("mt564-gemma-lora.zip")


  adding: mt564-gemma-lora/ (stored 0%)
  adding: mt564-gemma-lora/chat_template.jinja (deflated 52%)
  adding: mt564-gemma-lora/adapter_model.safetensors (deflated 8%)
  adding: mt564-gemma-lora/tokenizer.model (deflated 51%)
  adding: mt564-gemma-lora/adapter_config.json (deflated 55%)
  adding: mt564-gemma-lora/tokenizer_config.json (deflated 96%)
  adding: mt564-gemma-lora/tokenizer.json (deflated 84%)
  adding: mt564-gemma-lora/README.md (deflated 66%)
  adding: mt564-gemma-lora/special_tokens_map.json (deflated 70%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>